# Assembler Program (For our use)

## Opening the assembly program

In [1]:
path = "E:\\Projects\\Vivado\\Vector-Processor-SOC\\SOC_V7\\SOC_V7.srcs\\programs\\"
# file_name = "multiplier_large.asm"
file_name = "program1.asm"

In [2]:
with open(path + file_name, 'r') as assembly_file:
    content = assembly_file.read()
    
print(content)

/*
    This is the first assembly program to test the GPU
    This program will get the result of (3A+2B)B - BA

    matrix A is at Mem Address 256
    matrix B is at Mem address 512
*/

section .text
global _start
_start:
    loadrf 256; // load matrix A to the register file
    addsub 256 768 0; // A + A --> Mem address 768
    addsub 768 768 0; // 2A + A --> Mem address 768 (At 768 now 3A)
    mul 512 1024; // B*A --> Mem address 1024
    loadrf 512; // load matrix B to register file
    addsub 512 1280 0; // B+B ---> Mem address 1280
    loadrf 1280; // load 2B to register file
    addsub 768 768 0; // 3A + 2B --> 768 (At 768 now 3A + 2B)
    loadrf 512; // load B to the register file
    mul 768 768; // (3A+2B)B --> Mem address 768
    loadrf 1024; // load BA to the register file
    addsub 768 1024 2; // (3A+2B)B - BA
    nop;
    nop;
    term;



## Defining OPCODES

In [3]:
def addsub(A_ptr, Ans_ptr, opr):
    OPCODE = 15
    result1 = bin(OPCODE)[2:].zfill(8) # adding the opcode
    result2 = bin(opr)[2:].zfill(2) 
    result3 = bin(A_ptr)[2:].zfill(11)
    result4 = bin(Ans_ptr)[2:].zfill(11)
    
    final_code = hex(int(result1+result2+result3+result4, 2))[2:].zfill(8)
    
#     print(result1,result2,result3,result4)
#     print(final_code)
    
    return final_code

def mul(A_ptr, Ans_ptr):
    OPCODE = 12
    
    result1 = bin(OPCODE)[2:].zfill(8) # adding the opcode
    result2 = "00"
    result3 = bin(A_ptr)[2:].zfill(11)
    result4 = bin(Ans_ptr)[2:].zfill(11)
    
    final_code = hex(int(result1+result2+result3+result4, 2))[2:].zfill(8)
    
#     print(result1,result2,result3,result4)
#     print(final_code)
    
    return final_code

def nop():
    OPCODE = 0
    result1 = bin(OPCODE)[2:].zfill(8) # adding the opcode
    result2 = "".zfill(24)
    
    final_code = hex(int(result1+result2, 2))[2:].zfill(8)
    
#     print(result1, result2)
#     print(final_code)
    
    return final_code

def term():
    OPCODE = 1
    result1 = bin(OPCODE)[2:].zfill(8) # adding the opcode
    result2 = "".zfill(24)
    
    final_code = hex(int(result1+result2, 2))[2:].zfill(8)
    
#     print(result1, result2)
#     print(final_code)
    
    return final_code

def loadrf(A_ptr):
    OPCODE = 4
    result1 = bin(OPCODE)[2:].zfill(8) # adding the opcode
    result2 = bin(A_ptr)[2:].zfill(24)
    
    final_code = hex(int(result1+result2, 2))[2:].zfill(8)
    
#     print(result1,result2)
#     print(final_code)
    
    return final_code

# def loadrf(A_ptr, transpose):
#     OPCODE = 4
#     result1 = bin(OPCODE)[2:].zfill(8) # adding the opcode
#     result2 = bin(transpose)[2:].zfill(1)
#     result3 = bin(A_ptr)[2:].zfill(23)
    
#     final_code = hex(int(result1+result2+result3, 2))[2:].zfill(8)
    
# #     print(result1,result2,result3)
# #     print(final_code)
    
#     return final_code

def loadrow(ptr):
    OPCODE = 8
    result1 = bin(OPCODE)[2:].zfill(8) # adding the opcode
    result2 = bin(ptr)[2:].zfill(24)
    
    final_code = hex(int(result1+result2, 2))[2:].zfill(8)
    
#     print(result1,result2)
#     print(final_code)
    
    return final_code

In [48]:
def clean_assembly(assembly_code):
    # Find the index of '_start:'
    start_index = assembly_code.index('_start:')

    # Get the content after '_start:'
    code_after_start = assembly_code[start_index + 1:]

    # Remove comments and strip whitespace
    cleaned_code = [line.split(';')[0].strip() for line in code_after_start]

    # Remove empty lines
    cleaned_code = list(filter(None, cleaned_code))

    return cleaned_code

In [49]:
def execute_function(func_name, args):
    # Get the function by name
    func = globals().get(func_name)

    # Check if the function exists
    if func is not None and callable(func):
        # Execute the function with the provided arguments
        result = func(*args)
        return result
    else:
        print(f"Function '{func_name}' not found or not callable.")

In [50]:
ins = clean_assembly(content.split("\n"))

## Generating Machine Code (FOR TEST BENCH)

In [51]:
for i, instruction in enumerate(ins):
    instruction = instruction.split()
    
    opcode = instruction[0]
    args = [int(arg) for arg in instruction[1:]]
    
    mc = execute_function(opcode, args)
    
    #  @(posedge clk) #1 MemInAddr  = 12'h000; MemInData = 32'h04000400;
    
    print(f"@(posedge clk) #1 MemInAddr  = 12'h{hex(i)[2:].zfill(3)}; MemInData = 32'h{mc};")

@(posedge clk) #1 MemInAddr  = 12'h000; MemInData = 32'h04000100;
@(posedge clk) #1 MemInAddr  = 12'h001; MemInData = 32'h0f080300;
@(posedge clk) #1 MemInAddr  = 12'h002; MemInData = 32'h0f180300;
@(posedge clk) #1 MemInAddr  = 12'h003; MemInData = 32'h0c100400;
@(posedge clk) #1 MemInAddr  = 12'h004; MemInData = 32'h04000200;
@(posedge clk) #1 MemInAddr  = 12'h005; MemInData = 32'h0f100500;
@(posedge clk) #1 MemInAddr  = 12'h006; MemInData = 32'h04000500;
@(posedge clk) #1 MemInAddr  = 12'h007; MemInData = 32'h0f180300;
@(posedge clk) #1 MemInAddr  = 12'h008; MemInData = 32'h04000200;
@(posedge clk) #1 MemInAddr  = 12'h009; MemInData = 32'h0c180300;
@(posedge clk) #1 MemInAddr  = 12'h00a; MemInData = 32'h04000400;
@(posedge clk) #1 MemInAddr  = 12'h00b; MemInData = 32'h0f980400;
@(posedge clk) #1 MemInAddr  = 12'h00c; MemInData = 32'h00000000;
@(posedge clk) #1 MemInAddr  = 12'h00d; MemInData = 32'h00000000;
@(posedge clk) #1 MemInAddr  = 12'h00e; MemInData = 32'h01000000;


## Generating Machine Code (For VPU implemented in FPGA)

In [52]:
instrucntion_list = []
for i, instruction in enumerate(ins):
    instruction = instruction.split()
    
    opcode = instruction[0]
    args = [int(arg) for arg in instruction[1:]]
    
    mc = int(execute_function(opcode, args), 16)
    
    #  @(posedge clk) #1 MemInAddr  = 12'h000; MemInData = 32'h04000400;
    
#     print(f"@(posedge clk) #1 MemInAddr  = 12'h{hex(i)[2:].zfill(3)}; MemInData = 32'h{mc};")
    instrucntion_list.append(mc)
    

In [53]:
hex_ins = []
for instruction in instrucntion_list:
    temp = "0x{}".format(hex(instruction)[2:].zfill(8)) 
    hex_ins.append(temp)

In [54]:
c_arr = "{" + ",".join(hex_ins) + "}"
print(c_arr)

{0x04000100,0x0f080300,0x0f180300,0x0c100400,0x04000200,0x0f100500,0x04000500,0x0f180300,0x04000200,0x0c180300,0x04000400,0x0f980400,0x00000000,0x00000000,0x01000000}
